In [1]:
import pandas as pd

df = pd.read_csv("./data/raw/train_cax.csv")
oof = pd.read_csv("./data/result/oof.csv")

dt = pd.read_csv("./data/raw/test_cax.csv")
ds = pd.read_csv("./data/result/LSTM_submit.csv")

dt["label"] = list(ds["label"])
df["pred"] = oof["pred_label"]

In [2]:
from math import floor

df["leak"] = df["GR"]*100
df["leak"] = df["leak"] - df["leak"].apply(floor)
df["leak"] = df["leak"].apply(lambda x: round(x, 11))

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["label"]==i]["leak"].value_counts().keys())
        b = set(df[df["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))

0 1 	 1860 174921 	 0
0 2 	 1860 16100 	 0
0 3 	 1860 155569 	 0
0 4 	 1860 154186 	 0
1 2 	 174921 16100 	 0
1 3 	 174921 155569 	 0
1 4 	 174921 154186 	 0
2 3 	 16100 155569 	 0
2 4 	 16100 154186 	 0
3 4 	 155569 154186 	 0


In [3]:
s = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["pred"]==i]["leak"].value_counts().keys())
        b = set(df[df["pred"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        s |= a&b
        
len(s)

0 1 	 2019 175576 	 168
0 2 	 2019 16741 	 9
0 3 	 2019 156497 	 39
0 4 	 2019 155072 	 45
1 2 	 175576 16741 	 301
1 3 	 175576 156497 	 711
1 4 	 175576 155072 	 706
2 3 	 16741 156497 	 417
2 4 	 16741 155072 	 468
3 4 	 156497 155072 	 539


3135

In [4]:
1-len(df[df["label"]!=df["pred"]])/len(df)

0.9951895454545454

In [5]:
from tqdm import tqdm_notebook as tqdm
d = dict()
for c in tqdm(list(s)):
    k = df[df["leak"]==c]["pred"].value_counts().keys()[0]
    d[c] = k

In [6]:
res = []
for k, v in tqdm(zip(df["leak"], df["pred"]), total=len(df)):
    if k in d:
        res.append(d[k])
    else:
        res.append(v)

In [7]:
df["res"] = res
print(1-len(df[df["label"]!=df["res"]])/len(df))

0.9960870454545454


In [8]:
dt["leak"] = dt["GR"]*100
dt["leak"] = dt["leak"] - dt["leak"].apply(floor)
dt["leak"] = dt["leak"].apply(lambda x: round(x, 11))

ss = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(dt[dt["label"]==i]["leak"].value_counts().keys())
        b = set(dt[dt["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        ss |= a&b
        
len(ss)

0 1 	 1942 89413 	 92
0 2 	 1942 8422 	 4
0 3 	 1942 78105 	 22
0 4 	 1942 80352 	 12
1 2 	 89413 8422 	 125
1 3 	 89413 78105 	 253
1 4 	 89413 80352 	 229
2 3 	 8422 78105 	 142
2 4 	 8422 80352 	 149
3 4 	 78105 80352 	 214


1126

In [9]:
dd = dict()
for c in tqdm(list(ss)):
    k = dt[dt["leak"]==c]["label"].value_counts().keys()[0]
    dd[c] = k

In [10]:
res = []
for k, v in tqdm(zip(dt["leak"], dt["label"]), total=len(dt)):
    if k in dd:
        res.append(dd[k])
    else:
        res.append(v)
        
dt["label"] = res

In [11]:
dt[["unique_id","label"]].to_csv("./data/result/processed.csv", index=False)

In [65]:
from random import choice

idx = df[df["label"]!=df["res"]].index
k = choice(list(idx))

df.iloc[k-10:k+10,:]

,row_id,well_id,GR,label,pred,leak,res
2126725,425,1933,145.880373,1,1,0.037278,1
2126726,426,1933,141.692931,1,1,0.293092,1
2126727,427,1933,154.785489,1,1,0.548906,1
2126728,428,1933,144.978047,1,1,0.804720,1
2126729,429,1933,158.190605,1,1,0.060534,1
2126730,430,1933,147.193163,1,1,0.316348,1
2126731,431,1933,162.135722,1,1,0.572162,1
2126732,432,1933,104.948973,4,3,0.897272,2
2126733,433,1933,141.578973,4,3,0.897272,2
2126734,434,1933,143.368973,4,4,0.897272,2
